In [710]:
import random
import numpy as np

DATA = np.genfromtxt('Paridad.csv', delimiter=',')
# Define node types
NODE_TYPES = ['AND', 'OR', 'NOT', 'A', 'B', 'C']

# Define maximum depth of the tree
MAX_DEPTH = 3

class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

def and_func(x, y):
    return x and y

def or_func(x, y):
    return x or y

def not_func(x):
    return not x

def nor_func(x, y):
    return not (x or y)

def create_tree(depth):
    if depth <= 0 or random.random() < 0.1:
        # Terminal node, return either 0 or 1 randomly
        return Node(random.choice(['A', 'B', 'C']))
    else:
        # Non-terminal node, choose a function randomly
        node_type = random.choice(['AND', 'OR', 'NOT'])
        node = Node(node_type)
        # Recursively create children nodes
        for _ in range(2 if node_type in ['AND', 'OR'] else 1):
            node.children.append(create_tree(depth - 1))
        return node

def print_tree(node, indent=0):
    if node is not None:
        print('-' * indent + node.value)
        for child in node.children:
            print_tree(child, indent + 2)

def calcular_profundidades(nodo):
    if nodo.children == []:
        return 1
    else:
        return 1 + max([calcular_profundidades(child) for child in nodo.children])

def evaluar_arbol(arbol, a, b, c):
    # funcion recursiva que evalua el arbol, solo evalua para 1 renglon de la tabla de verdad
    if arbol.value == 'A':
        return a
    elif arbol.value == 'B':
        return b
    elif arbol.value == 'C':
        return c
    elif arbol.value == 'AND':
        return and_func(evaluar_arbol(arbol.children[0], a, b, c), evaluar_arbol(arbol.children[1], a, b, c))
    elif arbol.value == 'OR':
        return or_func(evaluar_arbol(arbol.children[0], a, b, c), evaluar_arbol(arbol.children[1], a, b, c))
    elif arbol.value == 'NOT':
        return not_func(evaluar_arbol(arbol.children[0], a, b, c))
    else:
        raise ValueError('Invalid node value')
    
def evaluar_arbol_completo(arbol):
    # funcion que evalua el arbol para todas las combinaciones de la tabla de verdad
    correctas = 0
    incorrectas = 0
    for i in DATA:
        prediccion = evaluar_arbol(arbol, i[0], i[1], i[2])
        # print("Prediccion: ", prediccion)
        # print("Respuesta esperada: ", i[3])
        if prediccion == i[3]:
            correctas += 1
        else:
            incorrectas += 1
    fitness = correctas/(correctas+incorrectas)
    # print("Correctas: ", correctas)
    # print("Incorrectas: ", incorrectas)
    # print("Porcentaje de acierto: ",fitness)
    return fitness

def mutacion(arbol, prob=0.1):
    # funcion que muta un arbol
    # crea un arbol nuevo
    # recorre el arbol original y muta con probabilidad 0.1
    # Mutacion sustituye el nodo por el arbol creado
    
    if random.random() < prob:
        prob = 0
        return create_tree(MAX_DEPTH)
    else:
        nuevo_arbol = Node(arbol.value)
        for child in arbol.children:
            nuevo_arbol.children.append(mutacion(child))
        return nuevo_arbol

def cruzamiento(arbol1, arbol2, prob = 0.8):
    # Identifica los puntos de cruza en cada padre
    # intercambiar los nodos entre los padres
    # retorna 2 hijos
    if random.random() > prob:
        return arbol2, arbol1
    else:
        nodos_a1 = num_nodos(arbol1)
        nodos_a2 = num_nodos(arbol2)
        subarbol1 = encontrar_subarbol(arbol1)
        subarbol2 = encontrar_subarbol(arbol2)
        hijo1 = insertar_subarbol(arbol1, subarbol2, nodos_a1)
        hijo2 = insertar_subarbol(arbol2, subarbol1, nodos_a2)
        return hijo1, hijo2
    




def collect_nodes_iterative(root):
    if not root:
        return []
    
    node_list = []
    stack = [root]

    while stack:
        node = stack.pop()
        node_list.append(node)
        stack.extend(node.children)

    return node_list

def encontrar_subarbol(root):
    if not root:
        return None
    
    # Collect all nodes in the tree
    all_nodes = collect_nodes_iterative(root)

    # Select a random node
    random_node = random.choice(all_nodes)

    # Create subtree from the random node
    return create_subtree_recursive(random_node)

def create_subtree_recursive(node):
    if not node:
        return None

    # Create a new node for the subtree
    subtree_root = Node(node.value)

    # Recursively add children to the subtree
    for child in node.children:
        subtree_child = create_subtree_recursive(child)
        subtree_root.add_child(subtree_child)

    return subtree_root

# def cruzar(arbol, subarbol, num_nodos):
#     arbol_cruzado = insertar_subarbol(arbol, subarbol, num_nodos)
#     return arbol_cruzado


# def encontrar_subarbol(arbol, num_nodos):
#     print(num_nodos)
#     prob = 2/num_nodos
#     if arbol.children == [] and num_nodos<2:
#         print("llegamos ak ", arbol.value)
#         return arbol
#     else:
#         for hijo in arbol.children:
#             if random.random() < prob:
#                 print("IF")
#                 return hijo
#             else:
#                 print("entro al else", hijo.value)
#                 return encontrar_subarbol(hijo, num_nodos-1) 
            
# def encontrar_subarbol(arbol, num_nodos):
#     prob = 1/num_nodos
#     if arbol is not None:
#         for hijo in arbol.children:
#             if random.random() < prob:
#                 subarbol = hijo
#                 break
#             else:
#                 return encontrar_subarbol(hijo, num_nodos-1)
#     return subarbol

# def collect_nodes(node, node_list):
#     if not node:
#         return
#     node_list.append(node)
#     for child in node.children:
#         collect_nodes(child, node_list)

# def encontrar_subarbol(root):
#     if not root:
#         return None
    
#     # Collect all nodes in the tree
#     all_nodes = []
#     collect_nodes(root, all_nodes)
#     # Select a random node
#     random_node = random.choice(all_nodes)

#     # Create subtree from the random node
#     return encontrar_subarbol(random_node)
  
def insertar_subarbol(arbol, subarbol, num_nodos):
    prob = 3/num_nodos
    if arbol.children == []:
        return subarbol
    else:
        for i, hijo in enumerate(arbol.children):
            if random.random() < prob:
                arbol.children[i] = subarbol
                return arbol
            else:
                return insertar_subarbol(hijo, subarbol, num_nodos-1)

def num_nodos(arbol):
    if arbol.children == []:
        return 1
    else:
        return 1 + sum([num_nodos(child) for child in arbol.children])
    



# Create a tree with maximum depth of 3
individual = create_tree(3)
print_tree(individual)
print(calcular_profundidades(individual))


A
1


In [337]:
padre1 = create_tree(2)
padre2 = create_tree(2)
print_tree(padre1)
print("\n")
print_tree(padre2)



AND
--OR
----B
----C
--AND
----B
----B


OR
--OR
----C
----B
--AND
----C
----B


In [99]:
hijo1, hijo2 = cruzamiento(padre1, padre2, 1)
print_tree(hijo1)
print("\n")
print_tree(hijo2)

OR
--B
--B


OR
--OR
----B
----B
--NOT
----A


In [537]:
print_tree(padre2)

OR
--OR
----C
----B
--AND
----C
----B


In [587]:
a = encontrar_subarbol(padre2)
print_tree(a)

[<__main__.Node object at 0x119fff140>, <__main__.Node object at 0x1190b83b0>, <__main__.Node object at 0x1190b8110>, <__main__.Node object at 0x1190b9a30>, <__main__.Node object at 0x1190bbdd0>, <__main__.Node object at 0x1190b9fa0>, <__main__.Node object at 0x1190b9070>]
[<__main__.Node object at 0x1190b83b0>, <__main__.Node object at 0x1190b8110>, <__main__.Node object at 0x1190b9a30>]
[<__main__.Node object at 0x1190b83b0>, <__main__.Node object at 0x1190b8110>, <__main__.Node object at 0x1190b9a30>]


RecursionError: maximum recursion depth exceeded

In [434]:
print_tree(hijo1)

Reporte TELCEL: 57151908

SyntaxError: invalid syntax (1378121544.py, line 3)

In [324]:
hijo1.children[0].children == []

True

In [273]:
num_nodos(hijo1)

3

In [342]:
print_tree(padre1)

AND
--OR
----B
----C
--AND
----B
----B


In [343]:
print_tree(padre2)

OR
--OR
----C
----B
--AND
----C
----B


In [350]:
arbol_nuevo = insertar_subarbol(padre1, padre2, num_nodos(padre1))

In [351]:
print_tree(arbol_nuevo)

OR
--OR
----C
----B
--AND
----C
----B


In [630]:
import random

class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child_node):
        self.children.append(child_node)

def collect_nodes_iterative(root):
    if not root:
        return []
    
    node_list = []
    stack = [root]

    while stack:
        node = stack.pop()
        node_list.append(node)
        stack.extend(node.children)

    return node_list

def create_subtree_from_random_node(root):
    if not root:
        return None
    
    # Collect all nodes in the tree
    all_nodes = collect_nodes_iterative(root)

    # Select a random node
    random_node = random.choice(all_nodes)

    # Create subtree from the random node
    return create_subtree_recursive(random_node)

def create_subtree_recursive(node):
    if not node:
        return None

    # Create a new node for the subtree
    subtree_root = TreeNode(node.value)

    # Recursively add children to the subtree
    for child in node.children:
        subtree_child = create_subtree_recursive(child)
        subtree_root.add_child(subtree_child)

    return subtree_root

# Example usage:
root = TreeNode("1")
child1 = TreeNode("2")
child2 = TreeNode("3")
child3 = TreeNode("4")
root.add_child(child1)
root.add_child(child2)
child1.add_child(child3)

random_subtree = create_subtree_from_random_node(root)


In [631]:
print_tree(random_subtree)

2
--4


In [655]:
random_subtree = create_subtree_from_random_node(padre2)
print_tree(random_subtree)

C


In [666]:
padre1 = create_tree(3)
padre2 = create_tree(3)
print_tree(padre1)
print("\n")
print_tree(padre2)


AND
--AND
----OR
------B
------A
----AND
------A
------C
--C


AND
--OR
----AND
------B
------A
----B
--NOT
----AND
------B
------A


In [694]:
hijo1, hijo2 = cruzamiento(padre1, padre2, 1)

In [695]:
print_tree(hijo1)
print("\n")
print_tree(hijo2)

AND
--NOT
----AND
------B
------A
--C


AND
--C
--NOT
----AND
------B
------A


In [687]:
# abrir archivo Paridad.csv sin header

import numpy as np

data = np.genfromtxt('Paridad.csv', delimiter=',')



print(data)

[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 1. 1.]
 [1. 0. 0. 0.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]
 [1. 1. 1. 0.]]


In [700]:
correctas = 0
incorrectas = 0
for i in data:
    prediccion = evaluar_arbol(hijo1, i[0], i[1], i[2])
    print("Prediccion: ", prediccion)
    print("Respuesta esperada: ", i[3])
    if prediccion == i[3]:
        correctas += 1
    else:
        incorrectas += 1
print("Correctas: ", correctas)
print("Incorrectas: ", incorrectas)
print("Porcentaje de acierto: ", correctas/(correctas+incorrectas))
    

Prediccion:  0.0
Respuesta esperada:  1.0
Prediccion:  1.0
Respuesta esperada:  0.0
Prediccion:  0.0
Respuesta esperada:  0.0
Prediccion:  1.0
Respuesta esperada:  1.0
Prediccion:  0.0
Respuesta esperada:  0.0
Prediccion:  1.0
Respuesta esperada:  1.0
Prediccion:  False
Respuesta esperada:  1.0
Prediccion:  False
Respuesta esperada:  0.0
Correctas:  5
Incorrectas:  3
Porcentaje de acierto:  0.625


In [702]:
evaluar_arbol_completo(hijo1)

Prediccion:  0.0
Respuesta esperada:  1.0
Prediccion:  1.0
Respuesta esperada:  0.0
Prediccion:  0.0
Respuesta esperada:  0.0
Prediccion:  1.0
Respuesta esperada:  1.0
Prediccion:  0.0
Respuesta esperada:  0.0
Prediccion:  1.0
Respuesta esperada:  1.0
Prediccion:  False
Respuesta esperada:  1.0
Prediccion:  False
Respuesta esperada:  0.0
Correctas:  5
Incorrectas:  3
Porcentaje de acierto:  0.625


0.625

In [711]:
def seleccionar_padres_ruleta(fitness):
    # seleccionar padres con probabilidad proporcional al fitness
    suma_fitness = sum(fitness)
    probabilidad = [fit/suma_fitness for fit in fitness]
    padres = []
    for _ in range(2):
        r = random.random()
        acumulado = 0
        for i, prob in enumerate(probabilidad):
            acumulado += prob
            if r < acumulado:
                padres.append(i)
                break
    return padres


def correr_algoritmo(generaciones, max_profundidad, poblacion=100):
    # crear poblacion inicial
    poblacion = [create_tree(max_profundidad) for _ in range(poblacion)]
    # evaluar poblacion
    fitness = [evaluar_arbol_completo(individual) for individual in poblacion]
    for generacion in range(generaciones):
        # seleccionar padres
        padres = seleccionar_padres_ruleta(fitness)
        # cruzar padres
        hijo1, hijo2 = cruzamiento(poblacion[padres[0]], poblacion[padres[1]])
        # mutar hijos
        hijo1 = mutacion(hijo1)
        hijo2 = mutacion(hijo2)
        # evaluar hijos
        fitness_hijo1 = evaluar_arbol_completo(hijo1)
        fitness_hijo2 = evaluar_arbol_completo(hijo2)
        # seleccionar individuos a reemplazar
        reemplazo = np.argmin(fitness)
        if fitness_hijo1 > fitness[reemplazo]:
            poblacion[reemplazo] = hijo1
            fitness[reemplazo] = fitness_hijo1
        reemplazo = np.argmin(fitness)
        if fitness_hijo2 > fitness[reemplazo]:
            poblacion[reemplazo] = hijo2
            fitness[reemplazo] = fitness_hijo2
    return poblacion, fitness




In [718]:
poblacion, fitness = correr_algoritmo(90000, 5)

In [724]:
fitness

[0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875,
 0.875]

In [725]:
print_tree(poblacion[np.argmax(fitness)])
accuracy = evaluar_arbol_completo(poblacion[np.argmax(fitness)])

AND
--OR
----B
----AND
------A
------C
--AND
----AND
------NOT
--------NOT
----------OR
------------OR
--------------B
--------------B
------------AND
--------------A
--------------C
------OR
--------C
--------B
----OR
------NOT
--------AND
----------B
----------C
------NOT
--------AND
----------OR
------------NOT
--------------AND
----------------C
----------------A
------------A
----------A


In [726]:
accuracy

0.75